In [1]:
!pip install -q accelerate==0.21.0
!pip install peft==0.4.0
!pip install transformers==4.31.0
!pip install trl==0.4.7
!pip install bitsandbytes==0.40.2 ## its used for quantization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.13.2
    Uninstalling peft-0.13.2:
      Successfully uninstalled peft-0.13.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

Import the Necessary Libraries

In [2]:
import os
import torch
from datasets import load_dataset ## what type of pre trained of datasets import it
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig,HfArgumentParser,TrainingArguments
from peft import LoraConfig,PeftModel
from trl import SFTTrainer ## SFT-Supervised Fine Tuning

In [3]:
dataset=load_dataset("timdettmers/openassistant-guanaco")
## They cover a range of conversational data,such as use queries,assistant responses,FAQs and context specific interactions.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


openassistant_best_replies_train.jsonl:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

openassistant_best_replies_eval.jsonl:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

In [4]:
dataset=dataset['train'].shuffle(seed=42).select(range(1000))
## This accesses the training split of the dataset.randomly shuffle the datasets,after shuffling,this selects the first 1000 examples from the datasets.

In [ ]:
## Pre-Processing the text :

In [5]:
from datasets import load_dataset
import re
# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)


Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
dataset_name='mlabonne/guanaco-llama2-1k'

In [7]:
model_name='NousResearch/Llama-2-7b-chat-hf'

In [8]:
new_model='Llama-2-7b-chat-finetune'

In [9]:
dataset=load_dataset(dataset_name,split='train')

README.md:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

(…)-00000-of-00001-9ad84bb9cf65a42f.parquet:   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
compute_dtype=getattr(torch, 'float16')  ## load tokenizer and model with QLORA Configuration
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    use_4bit=True
)

In [11]:
use_4bit=True

In [12]:
output_dir='./results' ## directory to save model checkpoints and logs

In [13]:
# check GPU Compatibility with bfloat16 use pytorch

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


# Load the base Model(The task where a model predicts the next token in asequence given the preceding tokens)
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto'
)

model.config.use_cache=False
model.config.pretraining_tp=1

# Load Llama tokenizer

tokenizer=AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'

## load Lora Configuration
peft_config=LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias='none',
    task_type='CAUSAL_LM'
)

# set training parameters

training_aruguments=TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim='paged_adamw_32bit',
    save_steps=10,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    report_to='tensorboard',
    lr_scheduler_type='cosine',
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    max_steps=100
)

# set supervised fine tuning parameters(fine tune only a subset of model parameters to save compute and memory)

trainer=SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    packing=True,
    args=training_aruguments
)

# Train the model

trainer.train()
## after training is complete the method returns the final training results,including metrics like the final loss.



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:268: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer,

Step,Training Loss
10,1.839500
20,1.495800
30,1.419500
40,1.349300
50,1.378800
60,1.451100
70,1.353400
80,1.315100
90,1.382900
100,1.460400


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=100, training_loss=1.4445725250244141, metrics={'train_runtime': 1529.2397, 'train_samples_per_second': 0.262, 'train_steps_per_second': 0.065, 'total_flos': 8362955636736000.0, 'train_loss': 1.4445725250244141, 'epoch': 0.4})

In [14]:
# save trainer model
trainer.model.save_pretrained(new_model)

In [19]:
from transformers import logging as transformers_logging
## hugging face transformers library used to show detailed messages

In [21]:
from transformers import pipeline

In [22]:
## ignore warnings
transformers_logging.set_verbosity_error()

## Run text generation pipeline with our next model

prompt='what is large language model?'
pipe=pipeline(task='text-generation',model=model,tokenizer=tokenizer,max_length=200)
result=pipe(f'<s>[INST] {prompt} [/INST]')
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=Tr

<s>[INST] what is large language model? [/INST] A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. everybody is talking about the latest language models, but what exactly are they? what are they used for? and how do they work? in this article, we will explore the basics of large language models and how they are changing the way we interact with technology.

what is a large language model?

a large language model is a type of neural network that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. the model is trained on a large dataset of text, such as books, articles, and websites, and is designed to generate text that is similar to the training data. the model is trained on a large dataset of text, such
